In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
import os
import sys
sys.path.append("../lib")
from dataUtils import *
import cPickle as pickle
import cv2
from collections import Counter
from scipy.interpolate import UnivariateSpline
from scipy.stats import entropy 
from scipy.stats import chisquare
from scipy.stats import ttest_ind


In [2]:
popular_color_features = "../Logs/vine_features_ordered.csv"
unpopular_color_features = "../Logs/unpopular_vine_features_ordered.csv"
aesthetic_color_features = "../Logs/aesthetic_features_6.csv"

In [3]:
def readFeatureFile(color_features):
    with open(color_features) as g:
        featureLines = g.readlines()
    print len(featureLines)
    return featureLines

def readAesFeatuers(lines):
    features = []
    for line in lines:
        components = line.strip().split('|')
        features.append([float(x) for x in components[1:]])
    return features

def readFeatures(lines):
    ids = []
    features = []
    feats = []
    
    components = lines[0].strip().split(',')
    ID = components[0].strip().split('_')[0]
    
    for line in lines:
        components = line.strip().split('|')
        postId = components[0].strip().split('_')[0]
        if postId != ID:
            ids.append(postId)
            ID = postId
            #Take median of all frames in a vine for all the attributes
            median = np.median(feats,0)
            for i in range(len(median)):
                if np.isnan(median[i]):
                    median[i] = 0.0
            features.append(median)
            feats[:] = []
            feats.append([float(x) for x in components[1:]]) 
        else :
            feats.append([float(x) for x in components[1:]]) 
            
    return features, ids
    

In [4]:
pop_features = readFeatureFile(popular_color_features)
unpop_features = readFeatureFile(unpopular_color_features)
aesthetic_features = readFeatureFile(aesthetic_color_features)

50187
5367
1149


In [5]:
popfeatures_matrix , popposts = readFeatures(pop_features)

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:3567: RuntimeWarning: Invalid value encountered in median for 3 results
  RuntimeWarning)


In [6]:
unpopfeatures_matrix , unpopposts = readFeatures(unpop_features)

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:3567: RuntimeWarning: Invalid value encountered in median for 1 results
  RuntimeWarning)


In [7]:
aes_feature = readAesFeatuers(aesthetic_features)

In [8]:
print len(unpopfeatures_matrix) , len(unpopposts)

2939 2939


In [ ]:
unpop_aes_features = np.asarray(unpopfeatures_matrix)
aes_feats = np.asarray(aes_feature)
pop_aes_features = np.asarray(popfeatures_matrix)

print unpop_aes_features.shape , aes_feats.shape , pop_aes_features.shape

In [ ]:
samples_pop = np.random.choice(pop_aes_features.shape[0],1000)
samples_unpop = np.random.choice(unpop_aes_features.shape[0], 1000)
samples_aes = np.random.choice(aes_feats.shape[0], 1000)
sampledpop_aes_features = pop_aes_features[samples_pop,:]
sampledunpop_aes_features = unpop_aes_features[samples_unpop,:]
sampled_aes = aes_feats[samples_aes,:]

In [ ]:
print sampledpop_aes_features.shape , sampledunpop_aes_features.shape

In [ ]:
num = 7
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampled_aes[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Rule of Thirds distribution for unpopular and popular videos', fontsize = 25)
plt.show()

print np.mean(sampledpop_aes_features[:,num]) , np.var(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.var(unpop_aes_features[:,num])
print np.mean(sampled_aes[:,num]) , np.var(sampled_aes[:,num])

In [ ]:
num = 4
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(unpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampled_aes[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic Image dataset'])
plt.title('Modified Luo Simplicity distribution for unpopular and popular videos', fontsize = 25)
plt.show()
print np.mean(sampledpop_aes_features[:,num]) , np.var(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.var(unpop_aes_features[:,num])
print np.mean(sampled_aes[:,num]) , np.var(sampled_aes[:,num])

In [ ]:
for i in range(sampledunpop_aes_features.shape[1]):
    print "Index %d" %i
    print "Aesthetic images: %0.3f , %0.3f , %0.3f "%( np.mean(aes_feats[:,i]), np.median(aes_feats[:,i]) , np.var(aes_feats[:,i]) )
    print "Popolar Vines: %0.3f , %0.3f , %0.3f "%( np.mean(sampledpop_aes_features[:,i]), np.median(sampledpop_aes_features[:,i]) , np.var(sampledpop_aes_features[:,i]) )
    print "Unpopular Vines: %0.3f , %0.3f , %0.3f "%( np.mean(sampledunpop_aes_features[:,i]), np.median(sampledunpop_aes_features[:,i]) , np.var(sampledunpop_aes_features[:,i]) )
    print "\n"
                                                                                                  

In [ ]:
print entropy(sampledpop_aes_features, sampled_aes, base=None)
print entropy(sampledunpop_aes_features, sampled_aes, base=None)


In [ ]:
num = 16
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0 )
plt.hist(unpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(aes_feats[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' ,'Aesthetic Image dataset'])
plt.title('Sharp pixel proportion distribution for popular , unpopular videos Vs Top aesthetic images', fontsize = 25)
plt.grid()
plt.show()
print np.mean(sampledpop_aes_features[:,num]) , np.var(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.var(unpop_aes_features[:,num])
print np.mean(sampled_aes[:,num]) , np.var(sampled_aes[:,num])

In [ ]:
print np.mean(sampledpop_aes_features[:,16]) , np.median(sampledpop_aes_features[:,16])
print np.mean(unpop_aes_features[:,16]) , np.median(unpop_aes_features[:,16])
print np.mean(aes_feats[:,16]) , np.median(aes_feats[:,16])

In [ ]:
fig, ax = plt.subplots()
num = 4
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 500, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 500, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(aes_feats[:,num], 500, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Rule of Thirds distribution for unpopular and popular videos', fontsize = 25)
plt.show()

In [ ]:
fig, ax = plt.subplots()
num = 3
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 100, normed=0,histtype='step', cumulative=False, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 100, normed=0,histtype='step', cumulative=False, linewidth = 1.0)
plt.hist(aes_feats[:,num], 100, normed=0,histtype='step', cumulative=False, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Luo Simplicity distribution for different samples', fontsize = 25)
plt.show()

print np.mean(sampledpop_aes_features[:,num]) , np.median(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.median(unpop_aes_features[:,num])
print np.mean(aes_feats[:,num]) , np.median(aes_feats[:,num])
t, p = ttest_ind( sampledpop_aes_features[:,num], sampled_aes[:,num])
print p
t, p = ttest_ind( sampledunpop_aes_features[:,num], sampled_aes[:,num])
print p


In [ ]:
fig, ax = plt.subplots()
num = 16
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(aes_feats[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Luo Simplicity distribution for different samples', fontsize = 25)
plt.show()

print np.mean(sampledpop_aes_features[:,num]) , np.median(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.median(unpop_aes_features[:,num])
print np.mean(aes_feats[:,num]) , np.median(aes_feats[:,num])